## **2. 기본적인 추천 시스템**
### 2.1 데이터 읽기
MovieLens 100K 데이터
- 사용자 데이터: u.user
- 영화 데이터: u.item
- 영화평가 파일: u.data

In [13]:
# u.user 파일 데이터프레임(df)으로 읽기
import pandas as pd
import numpy as np

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code'] #컬럼명 지정
users = pd.read_csv('u.user', sep = '|', names = u_cols, encoding = 'latin-1')
#텍스트 파일, 데이터 '|'으로 분리되어 있음

users = users.set_index('user_id') #user_id 기준으로 인덱스 지정
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [4]:
#u.item 파일 데이터프레임(df)으로 읽기
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
#장르: 19가지 변수로 표시 (0과 1로 표시)

movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
#u.data 파일 데이터프레임(df)으로 읽기
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1') 
#데이터 tab으로 분리되어 있음

ratings = ratings.set_index('user_id')
ratings.head()

#rating: 1~5점의 점수 척도
#timestamp: 평가한 연도/날짜/시간 숫자로 변환

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


### 2.2 인기제품 방식
: 개별 사용자에 대한 정보가 없는 경우/ 정확도에 관계없이 가장 간단한 추천을 제공하는 경우에 사용하는 방법<br>
▶ 모든 사람에게 똑같은 추천을 해주는 것 (가장 인기 있는 제품(best-seller))

In [8]:
#best-seller 추천
##각 제춤의 평점 평균화, 평균값 가장 높은 순서대로 추천
def recom_movie(n_items):
    movie_sort = movie_mean.sort_values(ascending = False)[:n_items]
    #.sort_values(ascending = False)[:n] = n개의 데이터 순서대로 내림차순 나열
    recom_movies = movies.loc[movie_sort.index]
    
    recommendations = recom_movies['title']
    #영화 제목 추천
    return recommendations

movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
#평점을 movie_id 기준으로 그룹핑, 평균값 구하기
#().groupby(['기준 컬럼'])[구하려는 컬럼].mean()

recom_movie(15)

movie_id
1293                                      Star Kid (1997)
1467                 Saint of Fort Washington, The (1993)
1653    Entertaining Angels: The Dorothy Day Story (1996)
814                         Great Day in Harlem, A (1994)
1122                       They Made Me a Criminal (1939)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1189                                   Prefontaine (1997)
1500                            Santa with Muscles (1996)
1536                                 Aiqing wansui (1994)
1449                               Pather Panchali (1955)
1642                             Some Mother's Son (1996)
119                Maya Lin: A Strong Clear Vision (1994)
1398                                          Anna (1996)
1594                                       Everest (1998)
Name: title, dtype: object

#### 함수 정의&설명
- movie_sort = movie_mean.sort_values(ascending = False)[:n_items]<br>
: 각 영화의 평점평균인 movie_mean을 내림차순 정렬, 상위부터 지정 개수(n_item)만큼 movie_sort에 저장
<br>

- recom_movies = movies.iloc[movie_sort.index]<br>
: movies에 있는 영화 중 선정된 n개 영화에 대한 정보 recom_movies에 저장
<br>

- recommendations = recom_movies['title']<br>
: 선정된 n개 영화의 영화 제목 골라내서 recommendations에 저장, 반환 
<br>

- movie_mean = ratings.groupby(['movie_id'])['rating'].mean()<br>
: ratings 평가 데이터를 groupby() 메소드 활용, 각 영화별 평점의 평균 구한뒤 movie_mean에 저장

In [10]:
#위 코드 한 줄로 요약
def recom_movie2(n_items):
    return movies.loc[movie_mean.sort_values(ascending = False)[:n_items].index]['title']

recom_movie2(15)

movie_id
1293                                      Star Kid (1997)
1467                 Saint of Fort Washington, The (1993)
1653    Entertaining Angels: The Dorothy Day Story (1996)
814                         Great Day in Harlem, A (1994)
1122                       They Made Me a Criminal (1939)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1189                                   Prefontaine (1997)
1500                            Santa with Muscles (1996)
1536                                 Aiqing wansui (1994)
1449                               Pather Panchali (1955)
1642                             Some Mother's Son (1996)
119                Maya Lin: A Strong Clear Vision (1994)
1398                                          Anna (1996)
1594                                       Everest (1998)
Name: title, dtype: object

### 2.3 추천 시스템의 정확도 측정
: 추천 시스템의 성능 = '정확성' <br>
▶ 추천 시스템이 계산한 선호도 예측치가 실제 선호도에 가까운지 판단

#### RMSE(Root Mean Squared Error)
: 각 예측값과 해당 실제값의 차이의 제곱의 평균의 제곱근

In [11]:
#정확도 계산 (RMSE 함수 정의)
    #y_true: 실제값의 벡터
    #y_pred: 예측값의 벡터

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [14]:
#실제값, best-seller 방식으로 구한 예측값의 RMSE = 0.996
rmse = [] #결과 저장할 rmse 변수 초기화

for user in set(ratings.index):
    y_true = ratings.loc[user]['rating']
    y_pred = movie_mean[ratings.loc[user]['movie_id']]
    accuracy = RMSE(y_true, y_pred)
    rmse.append(accuracy)
print(np.mean(rmse))
#구한 rmse값들의 평균값

0.996007224010567


#### 함수 정의&설명
- for user in set(ratings.index):<br>
: ratings에 있는 user_id를 다 추출한 뒤 한 user마다 for문 루프 돌림
<br>

- y_true = ratings.loc[user]['rating']<br>
: y_true(실제값)에 각 사용자가 평가한 모든 영화 평점 저장
<br>

- y_pred = movie_mean[ratings.loc[user]['movie_id']]<br>
: y_pred(예측값)에 best-seller 방식으로 구한 평점평균 저장(=해당 영화의 예측값)
<br>

- accuracy = RMSE(y_true, y_pred)<br>
: 실제값과 예측값을 활용한 한 유저의 RMSE 구함

### 2.4 사용자 집단별 추천
: 사용자들을 비슷한 특성의 사람들을 묶은 소집단으로 나눔 ▶ 각 집단의 평점평균 바탕으로 추천<br>
예) 성별, 나이, 직업

In [15]:
#데이터 읽어오기 (컬럼 다시 정의)
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1')

In [16]:
#데이터 전처리
#1) timestamp 제거
ratings = ratings.drop('timestamp', axis = 1)

#데이터프레임.drop('제거할 컬럼', axis = 1)

In [17]:
#2) movies 데이터프레임에서 movie_id, title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

In [18]:
#train - test set 분리
from sklearn.model_selection import train_test_split

x = ratings.copy() #ratings 데이터 복사(.copy()로 원본 보존)
y = ratings['user_id']
#train - test set 분리시 user_id 기준으로 하는 stratified sampling 방식 사용 (각 user_id별로 train, test set의 비율 동일하게 유지)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25, stratify = y)
#75%: train data, 25%: test data
#y_train, y_tes에는 사용자 id 저장

In [23]:
#RMSE 계산함수 정의
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

#모델별 RMSE 계산하는 함수 정의
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

#train data로 Full Matrix 구하기
rating_matrix = x_train.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')
rating_matrix.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1675,1676,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,NaN,3.0,NaN,5.0,4.0,1.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 함수 정의&설명
- id_pairs = zip(x_test['user_id'], x_test['movie_id'])<br>
: 예측 대상인 test set에 있는 사용자(user_id), 영화(movie_id) pair로 짝을 맞춘 데이터 형성
<br>

- y_pred = np.array([model(user, movie) for (user,movie) in id_pairs]<br>
: 사용자-영화 짝에 대해 주어진 예측 모델에 의한 예측값 계산, y_pred에 리스트로 저장
<br>

- y_true = np.array(x_test['rating'])<br>
: x_test에서 rating 실제 평점값 리스트 받아와 y_true에 리스트로 저장
<br>

- rating_matrix = x_train.pivot(index = 'user_id', columns = 'movie_id', values = 'rating')<br>
: pandas DataFrame의 pivot() 메소드 활용해서 train set의 full matrix 구함 ▶ 추천 알고리즘에서 사용
---

1. 예측은 한 사용자 - 한 영화 조합에 대해서 이루어진다.
2. 사용자의 특성에 따라 집단을 나누어 예측하기 위해서는 사용자 데이터(u.user)도 필요하다.
3. 평점 데이터, 사용자 데이터 합쳐야(merge)해야하므로 full matrix를 구해야한다.
- full matrix: 사용자-영화-평점으로 이루어진 데이터를 사용자, 영화 각 차원으로 하는 전체 array를 구하는 것
4. full matrix에는 모든 조합을 표시하기에 평가하지 않은 조합은 NaN으로 표시한다.

In [24]:
#전체 평균(best-seller 방식)으로 예측치 계산하는 기본 모델
def best_seller(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0 #평균값 없으면 3.0 반환
    return rating

train_mean = x_train.groupby(['movie_id'])['rating'].mean()
score(best_seller)

1.0212858883514089

#### 함수 정의&설명
- try ~ except<br>
: train set에는 없는데 test set에는 존재하는 영화로 인해 발생하는 오류 방지하기 위해 사용
<br>

- rating - train_mean[movie_id]<br>
: 저장해당 영화의 평균 데이터(train_mean) 존재하면 평균값으로 돌려줌
<br>

#### 성별 기준 추천

In [25]:
#Full Matrix - 사용자 데이터와 merge
merged_ratings = pd.merge(x_train, users)
users = users.set_index('user_id') #인덱스 user_id로 설정

In [26]:
#성별 별로 평점평균 계산

g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()
#movie_id, sex, rating 열만 남긴 뒤 성별 기준으로 평점 평균 계산해 g_mean에 저장

In [28]:
#성별 별로 평균 예측치로 돌려주는 함수 정의
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id]:
            gender_rating = g_mean[movie_id][gender]
        else:
            gender_rating = 3.0
    else:
        gender_rating = 3.0
    return gender_rating

score(cf_gender)

#사용자의 성별에 따른 추천이 정확도를 개선하지 못함

1.0305410484655932

#### 함수 정의&설명
- if gender in g_mean[movie_id]<br>
: 예측 대상 영화가 해당 성별의 평균 가지고 있는지 확인<br>
✅ 평가한 사용자가 적은 영화의 경우, gender_mean에 해당 영화가 없는 경우 있을 수 있음

### 연습문제
#### 2-1 사용자의 직업('occupation')에 따라 집단 나누어 예측값 구하는 함수 만들고, 정확도 계산

In [31]:
o_mean = merged_ratings[['movie_id', 'occupation', 'rating']].groupby(['movie_id', 'occupation'])['rating'].mean()

In [32]:
#사용자 직업별로 평균 예측치로 돌려주는 함수 정의
def cf_occupation(user_id, movie_id):
    if movie_id in rating_matrix:
        occupation = users.loc[user_id]['occupation']
        if occupation in o_mean[movie_id]:
            occupation_rating = o_mean[movie_id][occupation]
        else:
            occupation_rating = 3.0
    else:
        occupation_rating = 3.0
    return occupation_rating

score(cf_occupation)
#사용자 직업에 따른 추천은 정확도를 개선함(1.02에서 1.11로 상승)

1.118582875956232

#### 대괄호 .loc, .iloc, .ix 메서드의 차이
- .loc 메서드: 레이블 이용해 행 선택 ▶ 슬라이싱시 처음과 끝 둘 다 포함
- .iloc 메서드: 위치 정수 이용해 행 선택 ▶ 슬라이싱시 처음 포함, 끝 포함 안됨
- .ix 메서드: 레이블, 위치 정수 둘 다 사용가능함